In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

df1 = pd.read_csv('Datasets\\ALLFLOWMETER_HIKARI2021.csv')
df2 = pd.read_csv('Datasets\\ALLFLOWMETER_HIKARI2022.csv')

#df1.drop('bwd_last_window_size', axis=1)

# Combining two IDS datasets acquired over 2 years to develop good model
combined_df = pd.concat([df1, df2], ignore_index=True)

# we remove the 'flow_duration' here too because it has no real impact on the analysis and the formats are very different from same
# synthetic dataset reproduced as shown in paper.
combined_df = combined_df.drop(['Unnamed: 0', 'uid', 'flow_duration','bwd_last_window_size'], axis=1)

combined_df['attack_category'] = combined_df['attack_category'].str.strip()
# Replacing similar values with correct version
combined_df['attack_category']=combined_df['attack_category'].replace('Brutefoce', 'Bruteforce')

attack_category_counts = combined_df['attack_category'].value_counts()
print("Attack categories:",attack_category_counts)

y = combined_df['Label']

X = combined_df.drop(['Label', 'originh', 'responh'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

# Names of columns with missing values
cols_with_missing_values = [col for col in X_train.columns
                            if X_train[col].isnull().any()]

print("Columns with missings values:",cols_with_missing_values)
# With prelimnary analysis we found that there were no missing values in any 
# of the columns. Hence, it is a robust dataset.

#getting list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables")
print(object_cols)

